# Optical Pumping

Author: Dounan Du

Abstract: This is a numerical simulation of an idealized optical pumping process. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
import qutip
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, destroy,
                   expect, fock, fock_dm, mesolve, qeye, sigmax, sigmay,
                   sigmaz, tensor, thermal_dm)
import math

%matplotlib inline

## Introduction

In quantum optical experiments, especially in thoses involve cold atoms, the quality of the optical pumping preparation of the atomic states is key to a clear demonstration of the core physics. This numerical simulation provides some insight on how the pumping quality is dependent on various pumping parameters, such as pumping Rabi frequency, pumping duration.

## Create Rb87 atomic structure 

The atomic structure is encoded in two fundamental class: HyperfineLevel and HyperfineTransition class.

In [2]:
class HyperfineLevel:

    """
    Atomic hyperfine sublevels are represented by this class.

    Attributes:
        F (int): The hyperfine level quantum number.
        mF (int): The Zeeman sublevel quantum number.
        FineStruc (int): the number represent the Fine level as follows: 0 - 5S1/2; 1 - 5P1/2; 2 - 5P3/2.
        levelIndex(int): the index of python ket vector elements represent this hyperfine sub level.

    """
    def __init__(self, F, mF, FineStruc, levelIndex):
        self.F = F
        self.mF = mF
        self.FineStruc = FineStruc
        self.levelIndex = levelIndex

class HyperfineTransition:
    """
    Transitions between hyperfine Zeeman levels are represented by this class.

    Attributes:
        in_: represent initial hyperfine level, same numbering system as HyperfineLevel class.
        f_: represent final level.
        reducedDipoleM (float): the reduce dipole moment of the form <J||er||J'>. Value from Steck data.
        dipoleM (float): dipole moment in units of related reduced dipole moment, value from Steck data.
        Pol (int): Polarization of the transition, with -1 - sigma-; 0 - pi; +1 - sigma+
    
    """
    def __init__(self,in_FineStruc, in_F,in_mF,f_FineStruc,f_F,f_mF,reducedDipoleM,dipoleM,Pol):
        self.in_FineStruc = in_FineStruc
        self.in_F = in_F
        self.in_mF = in_mF
        self.f_FineStruc = f_FineStruc
        self.f_F = f_F
        self.f_mF = f_mF
        self.reducedDipoleM = reducedDipoleM
        self.dipoleM = dipoleM
        self.Pol = Pol


Now create a database to store the Steck Rb87 data as HyperfineTransition objects.

In [3]:
# this cell is a data base. Each enrty represent a hyperfine to hyperfine transition. Add relevent entry to the specific system.

rD02 = 3.584e-29 #87Rb reduced dipole matrix element <J=1/2||er||J'=3/2>

database = []

# 87Rb D2 5S1/2 -> 5P3/2, sigma+ transition, F = 1 -> F' =2
entry1 = [[-1,math.sqrt(1/24)],[0,math.sqrt(1/8)],[1,math.sqrt(1/4)]]
for i in entry1:
    database.append(HyperfineTransition(0,1,i[0],2,2,i[0]+1,rD02,i[1],1))

# 87Rb D2 5S1/2 -> 5P3/2, pi transition, F = 1 -> F' =2
entry2 = [[-1,-math.sqrt(1/8)],[0,-math.sqrt(1/6)],[1,-math.sqrt(1/8)]]
for i in entry2:
    database.append(HyperfineTransition(0,1,i[0],2,2,i[0],rD02,i[1],0))

# 87Rb D2 5S1/2 -> 5P3/2, sigma- transition, F = 1 -> F' =2
entry3 = [[-1,math.sqrt(1/4)],[0,math.sqrt(1/8)],[1,math.sqrt(1/24)]]
for i in entry3:
    database.append(HyperfineTransition(0,1,i[0],2,2,i[0]-1,rD02,i[1],-1))

# 87Rb D2 5S1/2 -> 5P3/2, sigma+ transition, F = 2 -> F' =2
entry4 = [[-2,math.sqrt(1/12)],[-1,math.sqrt(1/8)],[0,math.sqrt(1/8)],[1,math.sqrt(1/12)]]
for i in entry4:
    database.append(HyperfineTransition(0,2,i[0],2,2,i[0]+1,rD02,i[1],1))

# 87Rb D2 5S1/2 -> 5P3/2, pi transition, F = 2 -> F' =2
entry5 = [[-2,-math.sqrt(1/6)],[-1,-math.sqrt(1/24)],[0,0],[1,math.sqrt(1/24)],[2,math.sqrt(1/6)]]
for i in entry5:
    database.append(HyperfineTransition(0,2,i[0],2,2,i[0],rD02,i[1],0))

# 87Rb D2 5S1/2 -> 5P3/2, sigma- transition, F = 2 -> F' =2
entry6 = [[-1,-math.sqrt(1/12)],[0,-math.sqrt(1/8)],[1,-math.sqrt(1/8)],[2,-math.sqrt(1/12)]]
for i in entry6:
    database.append(HyperfineTransition(0,2,i[0],2,2,i[0]-1,rD02,i[1],-1))
    

Specify the fine level and hyperfine levels that are included in the system model, here the matrix represention is under the convention that in python numpy array, smaller index corresponds to lower energy levels. For example, for kets $\ket{e}$ and $\ket{g}$, the corresponding vector is np.array([[0],[1]]) and np.array([[1],[0]]).

The involved hyperlevels are specified by a np array. For example for the $5S_{1/2}\ F=1$ hyperfine level the identify column vector is np.array([[0,1]])

here input the involved levels. For example: $5S_{1/2}\ F=1,2$ and $5P_{3/2}\ F = 2$ (_change according to specific system_)

In [ ]:
#specify related hyperfine levels, order them as increacing energy
levelInvolved = np.array([[0,1],[0,2],[2,2]])

Define a function to automatically generate all sub-level objects from the provided involved levels, and generate a index list that encodes the structure information.

In [ ]:
def Gen_levelStruct(levelInvolved):

    """
    automatically generate all sub-level objects from the provided involved levels
    
    """

    n = (np.sum(levelInvolved,axis=0)[1])*2 + levelInvolved.shape[0] #find total dimension of the Hilbert space
    l = [None] * n #initialize the list that hold all sub hperfine level objects.

    j=0 #this for loop structure generate a list of all the involved level objects, and they are ordered as increacing of energy
    for i in range(levelInvolved.shape[0]): 
        subLvs = range(2*levelInvolved[i,1]+1)-levelInvolved[i,1]
        for hyLevels in subLvs:
            l[j] = HyperfineLevel(levelInvolved[i,1],hyLevels,levelInvolved[i,0],j)
            j+=1

    # creating a list of the atomic level index that encodes the level structure 
    lIndex = [None] * n
    for i in range(n):
        lIndex[i] = i

    segment_length = []
    for i in levelInvolved:
        segment_length.append(i[1] * 2 + 1)

    start = 0
    lStructure_list = []

    for length in segment_length:
        end = start + length
        lStructure_list.append(lIndex[start:end])
        start = end

    return l,lStructure_list

l,lStructure_list = Gen_levelStruct(levelInvolved)

## Generate initial density operator

Assume initially the atom pupolation is equally distributed over $5S_{1/2}\ F=2$ hyperfine Zeeman levels. 

In [ ]:
d_Hilbert = 13

rho_rf = 0
for i in range(3,8):
    rho_rf += qutip.fock_dm(d_Hilbert,i)*0.2

In rotating frame, the Hamiltonian is 
$$
\begin{equation}
\tilde{H} = \tilde{H}_0 + \tilde{H}_{AF}
\end{equation}
$$

